<a href="https://colab.research.google.com/github/marcellapbp/AnaliseEnem2019/blob/main/2_Visualizando_Distribui%C3%A7%C3%A3o_Inscritos_atrav%C3%A9s_Gr%C3%A1ficos_Geogr%C3%A1ficos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Visualizando Distribuição de Inscritos Nascidos em SP através de Gráficos Geográficos

**Fonte de apoio**
<br>https://www.kdnuggets.com/2020/09/geographical-plots-python.html
<br>https://joseotaviobiscaia.wixsite.com/geopdf/arquivo-das-cidades-em-excel
<br>http://metadados.inde.gov.br/geonetwork/srv/por/metadata.show?uuid=46e1b9ec-5ed7-4450-ac42-26f948130fa6
<br>https://plotly.com/python/choropleth-maps/


Introdução
---


A ideia desse experimento surgiu após a leitura do artigo do [KDNuggets](https://www.kdnuggets.com/2020/09/geographical-plots-python.html) junto  da curiosidade em saber qual a concentração dos inscritos no Enem 2019 nascidos no Estado de São Pulo em cada cidade. Com essa curiosidade em mente logo surgiu a primeira hipótese de que a concentração tende a ser maior próxima à capital, principalmente por desigualdade social, de forma que quem mora mais próximo a capital tem melhores condições de vida, maior interesse em participar do Enem e até maior facilidade no acesso a escolas no dia da prova, também com melhor acesso à internet para estudo entre outras questões.

---

Importando Bibliotecas


In [1]:
#para manipulação dos datasets
import pandas as pd
#para plotar o gráfico
import plotly.express as px

Realizando a leitura do dataset com dados dos Inscritos do Enem 2019
<br>
Essa amostra contém apenas inscritos nascidos no Estado de São Paulo

In [ ]:
df = pd.read_csv('/content/MICRODADOS_ENEM_2019_SP.csv')

Conferindo os dados se estão tabulados corretamente sem nenhuma inconsistência

In [3]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001017053,2019,3546801,Santa Isabel,35,SP,22,M,1,3,1,1506500.0,Santa Izabel do Pará,15.0,PA,1,4,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,EEACECDBADBEABAAEAEACAAAECBEEEBACACCDEDACADCB,ADBBE99999DEDDBCAAAEEDDDBAADCCACECDCAEBCBCEBBA...,EDBADBDBAECBBB*CBADBAECCBACECDEDECBACCECDCCAA,0.0,AAECACDEADCBCDDDBCBDADAEABCEBABEEBCBEECEBDADC,EEBCEEDBADBBCBABCCADCEBACDBBACCACACBEADBBADCB,ADBBEDCABAABBCBCDAAECDDDBAAAECADECDCEBDEEAECBD...,EEEADBEBACABCDBABECECACDCBDCCEDCDABEDECDDDBAA,1.0,160.0,200.0,180.0,200.0,200.0,940.0,H,E,C,B,7.0,C,A,B,C,A,A,B,A,B,A,A,A,A,C,A,A,D,A,B,B
1,190001019476,2019,3538204,Pinhalzinho,35,SP,19,M,1,3,1,1503804.0,Jacundá,15.0,PA,1,2,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,D,B,B,4.0,C,A,B,D,A,B,B,A,A,A,A,A,A,B,A,A,C,A,B,A
2,190001019850,2019,3550308,São Paulo,35,SP,17,F,1,3,1,1501402.0,Belém,15.0,PA,2,0,3,1.0,0,15567761.0,1501402.0,Belém,15.0,PA,4.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,BCDCABABABBCBAEEEBCACACAEDBCCAEBEABEADBAEACCB,BDABE99999BCBDADDBDCDAAECDEECDBCBCCDEDAACDEDBB...,DBEBAECBADCACECADBBBBCEBADCEABECAADDBEDDBAECB,0.0,DADCCEBBCCACBEEBEEBACBCDDDDADBCBBCEAEADEADAAE,BACCCBABBADCBCEEEBCACACEEDBCCADBEADBADBBBACDB,BDABEABCADBCBAADDBECDAAECDAECBECBCCDEEAAADDBBC...,DBEBACABCDBABECEEEDCBDCCEDCDABEDAADDDECACAECB,1.0,160.0,120.0,200.0,200.0,200.0,880.0,E,E,F,B,6.0,D,A,B,C,A,A,B,A,A,A,A,A,A,A,A,A,C,A,A,A
3,190001024874,2019,3548708,São Bernardo do Campo,35,SP,19,M,1,3,1,3548708.0,São Bernardo do Campo,35.0,SP,2,0,2,1.0,0,35904958.0,3548708.0,São Bernardo do Campo,35.0,SP,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,AC.BECDABAECBCBBEAACA.EDCDEAEAADCEABDAAACEBCC,99999CBEAEBBCEDBABEBDAADCEAEDEBBEACACACACDCEDB...,ADDCEEEDADABDBDAAACAECEBBABEBDBEEBADCEBDEDCDA,1.0,DADCCEBBCCACBEEBEEBACBCDDDDADBCBBCEAEADEADAAE,ACACEEBCCBABADBBBACDBBACCCCADCEBADCBEEDBBEADB,EBBADCABDABACBCEBDEEAAADDBECDECDDBADBCDAAECBCC...,DBEBACABCDBABECEEEDCBDCCEDCDABEDAADDDECACAECB,1.0,80.0,160.0,120.0,100.0,100.0,560.0,B,B,B,B,4.0,C,A,C,D,B,B,B,A,B,B,B,A,B,B,B,A,B,B,A,B
4,190001024931,2019,3549409,São Joaquim da Barra,35,SP,37,M,2,1,1,3549409.0,São Joaquim da Barra,35.0,SP,1,3,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,A,A,3.0,C,A,B,C,A,B,B,A,A,A,B,A,A,A,A,A,C,B,B,B


Realizando a leitura de um dataset encontrado no site do [INDE](http://metadados.inde.gov.br/geonetwork/srv/por/metadata.show?uuid=46e1b9ec-5ed7-4450-ac42-26f948130fa6) que contém a Geolocalização das Cidades do Brasil. Para esse experimento precisamos da latitude e longitude visto que alguns gráficos explorados durante este experimento só reconhecem nome de países ou no máximo os estados do EUA.

In [68]:
df_ibge = pd.read_csv('/content/BCIM_Cidade_P.csv')
df_ibge.head()

,FID,id_objeto,nome,nomeabrev,geometriaaproximada,geom
0,BCIM_Cidade_P.301380,301380,Pojuca,NaN,NaN,POINT (-38.337469081656856 -12.430728101952266)
1,BCIM_Cidade_P.300699,300699,Cruz,NaN,NaN,POINT (-40.17630489052276 -2.918518104716811)
2,BCIM_Cidade_P.301905,301905,Bonito,NaN,NaN,POINT (-47.30694235975195 -1.367829928859953)
3,BCIM_Cidade_P.302319,302319,Arês,NaN,NaN,POINT (-35.16284744400113 -6.1969810369166405)
4,BCIM_Cidade_P.302648,302648,Macaé,NaN,NaN,POINT (-41.785189555693314 -22.377294006185394)


Realizando a limpeza e estruturação das informações Latitude e Longitude

In [ ]:
#criando novas colunas no dataset
df_ibge['Lat'] = ''
df_ibge['Lon'] = ''

#para cada linha separa o conteúdo nas novas colunas e elimina os parênteses
i = 0
for conteudo in df_ibge.geom:
  lista = conteudo.split()
  df_ibge['Lon'][i]= lista[1].replace('(','')
  df_ibge['Lat'][i] = lista[2].replace(')','') 
  i = i+1

Conferindo resultado da limpeza e estruturação

In [39]:
df_ibge.head()

,FID,id_objeto,nome,nomeabrev,geometriaaproximada,geom,Lat,Lon
0,BCIM_Cidade_P.301380,301380,Pojuca,NaN,NaN,POINT (-38.337469081656856 -12.430728101952266),-12.430728101952266,-38.337469081656856
1,BCIM_Cidade_P.300699,300699,Cruz,NaN,NaN,POINT (-40.17630489052276 -2.918518104716811),-2.918518104716811,-40.17630489052276
2,BCIM_Cidade_P.301905,301905,Bonito,NaN,NaN,POINT (-47.30694235975195 -1.367829928859953),-1.367829928859953,-47.30694235975195
3,BCIM_Cidade_P.302319,302319,Arês,NaN,NaN,POINT (-35.16284744400113 -6.1969810369166405),-6.1969810369166405,-35.16284744400113
4,BCIM_Cidade_P.302648,302648,Macaé,NaN,NaN,POINT (-41.785189555693314 -22.377294006185394),-22.377294006185394,-41.785189555693314


Criando um novo dataframe com dados consolidados de quantidade de inscritos por cidade para a montagem do gráifco

In [40]:
d = {'Cidade': df.NO_MUNICIPIO_RESIDENCIA.value_counts().index,
     'Qtdes' : df.NO_MUNICIPIO_RESIDENCIA.value_counts().values}
df_count = pd.DataFrame(data=d)

In [41]:
df_count.head()

,Cidade,Qtdes
0,São Paulo,917
1,Guarulhos,102
2,Campinas,76
3,Ribeirão Preto,69
4,Osasco,67


In [71]:
print(f'O conjunto de dados possui {df_count.shape[0]} cidades do Estado de são Paulo')

O conjunto de dados possui 349 cidades do Estado de são Paulo


Eliminando colunas não utilizadas do dataset usado para complementar a geolocalização

In [43]:
df_ibge = df_ibge.drop(columns=['FID','id_objeto', 'nomeabrev','geometriaaproximada','geom'])
df_ibge.head()

,nome,Lat,Lon
0,Pojuca,-12.430728101952266,-38.337469081656856
1,Cruz,-2.918518104716811,-40.17630489052276
2,Bonito,-1.367829928859953,-47.30694235975195
3,Arês,-6.1969810369166405,-35.16284744400113
4,Macaé,-22.377294006185394,-41.785189555693314


Esse dataset possui cidades de todo Brasil

In [44]:
df_ibge.shape

(5812, 3)

Renomeando a coluna que será a chave para mesclar os datasets

In [45]:
df_ibge = df_ibge.rename(columns={'nome':'Cidade'})
df_ibge.head()

,Cidade,Lat,Lon
0,Pojuca,-12.430728101952266,-38.337469081656856
1,Cruz,-2.918518104716811,-40.17630489052276
2,Bonito,-1.367829928859953,-47.30694235975195
3,Arês,-6.1969810369166405,-35.16284744400113
4,Macaé,-22.377294006185394,-41.785189555693314


Resultado do merge entre os datasets

In [46]:
df_final = pd.merge(df_count,df_ibge, on='Cidade')
df_final.head()

,Cidade,Qtdes,Lat,Lon
0,Guarulhos,102,-23.454237227889976,-46.53374514741128
1,Campinas,76,-22.905822218783705,-47.066350129618975
2,Ribeirão Preto,69,-21.170391981881515,-47.81027519335388
3,Osasco,67,-23.53286880192161,-46.79195514213204
4,São José dos Campos,58,-23.190034056806446,-45.88450791544053


Foi notado que há alguma inconsistência nos dados, pois o resultado de linhas ficou maior do que era antes do merge. Provavelmente há cidades de outros estados que coincidem com o estado de São Paulo. No entanto esse dataset não possuía uma coluna identificando os estados, como se trata de um experimento, por hora foi desconsiderado e os dados foram utilizados dessa maneira mesmo.

In [47]:
df_final.shape

(385, 4)

Conferindo os valores mínimos e máximos e concentração da quantidade de inscritos para conferir se o mapa projetado faz sentido tendo em vista esses dados.

In [50]:
df_final.Qtdes.describe()

count    385.000000
mean       6.480519
std       11.675083
min        1.000000
25%        1.000000
50%        2.000000
75%        6.000000
max      102.000000
Name: Qtdes, dtype: float64

Conferindo total de inscritos

In [51]:
df_final.Qtdes.sum()

2495

Plotando o Mapa de Densidade


---

Nesse mapa foi informado a Latitude e Longitude, e o volume é a quantidades de inscritos, assim a mancha criada acima de cada posição no mapa fica maior de forma proporcional a quantidade de inscritos daquela localização.
<br>
O mapa foi centralizado e foi dado zoom de forma a centralizar o estado de São Paulo.


---

Por fim, mesmo com algumas inconsistências nos dados, é possível notar que de fato a hipótese inicial foi confirmada, a concentração de inscritos é maior nas regiões próximas à capital e na própria capital. No entanto a justificativa dessa hipótese de que isso se dá devido a desigualdade social não é possível sustentar apenas com os dados desse experimento, pois teríamos que ter dados socioeconômicos dos não participantes para efeito de comparação e validação dessa hipótese.

In [72]:
fig = px.density_mapbox(df_final, lat='Lat', lon='Lon', z='Qtdes', radius=40,
                        center=dict(lat=-22, lon=-48), zoom=5,
                        mapbox_style="stamen-terrain")
fig.show()